In [1]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.0 MB/s eta 0:00:0

In [30]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY_TK')

In [31]:
model = ChatOpenAI(api_key=key,model="gpt-3.5-turbo-0125")

# Spotify

In [ ]:
from langchain.tools import tool
from typing import List
import requests

# Optional: Use a dataclass to format results more clearly
def get_spotify_token(client_id: str, client_secret: str) -> str:
    """
    Fetches a bearer token from Spotify using client credentials.
    """
    url = 'https://accounts.spotify.com/api/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()['access_token']

import requests

@tool
def search_spotify_playlists(query: str) -> dict:
    """
    Search for Spotify playlists using a keyword (e.g., "sleep", "meditation").

    Returns a JSON object with the top 5 playlists including name, description, image, link, owner, and track count.
    """
    client_id = "key"
    client_secret = "key"

    try:
        token = get_spotify_token(client_id, client_secret)

        headers = {
            'Authorization': f'Bearer {token}'
        }
        params = {
            'q': query,
            'type': 'playlist',
            'limit': 5
        }
        url = 'https://api.spotify.com/v1/search'
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        playlists = response.json().get('playlists', {}).get('items', [])

        if not playlists:
            return {"message": "No playlists found for your query."}

        result = {
            "query": query,
            "total": len(playlists),
            "playlists": []
        }

        for pl in playlists:
            if pl is None:
                continue

            result["playlists"].append({
                "name": pl.get('name'),
                "description": pl.get('description'),
                "external_url": pl.get('external_urls', {}).get('spotify'),
                "image": pl.get('images', [{}])[0].get('url'),
                "owner": {
                    "name": pl.get('owner', {}).get('display_name'),
                    "url": pl.get('owner', {}).get('external_urls', {}).get('spotify')
                },
                "tracks": {
                    "url": pl.get('tracks', {}).get('href'),
                    "total": pl.get('tracks', {}).get('total')
                }
            })

        return result

    except Exception as e:
        return {"error": str(e)}

In [33]:
spotify_assistant = create_react_agent(
    model=model,
    tools=[search_spotify_playlists],
    prompt=(
        "You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists "
        "based on their search queries using the tool provided.\n\n"
        "🧠 Think step-by-step, and always call the `search_spotify_playlists` tool with the user query.\n"
        "📦 When returning results to the user, format the final output in **JSON** using this structure:\n\n"
        "{\n"
        '  "query": string,\n'
        '  "total": integer,\n'
        '  "playlists": [\n'
        "    {\n"
        '      "name": string,\n'
        '      "description": string,\n'
        '      "external_url": string,\n'
        '      "image": string,\n'
        '      "owner": {"name": string, "url": string},\n'
        '      "tracks": {"url": string, "total": integer}\n'
        "    },\n"
        "    ...\n"
        "  ]\n"
        "}\n\n"
        "Only return JSON. Do not add any extra commentary or text outside of the JSON structure."
    ),
    name="spotify_assistant"
)

# Sleep

In [36]:
import json
import re
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

class SleepCoachAgent:
    def __init__(self, api_key):
        self.llm = ChatOpenAI(openai_api_key=api_key, model="gpt-3.5-turbo-0125", max_tokens=4096)
        self.prompt = PromptTemplate(
    template = """
You are a smart, caring, and proactive Sleep Coach AI. Your role is to help users reflect on their sleep, understand behavioral patterns, and build sustainable habits for better rest and recovery.

Your job is to analyze the user's sleep-related input and provide a rich JSON response structured across multiple categories.

==============================
CONTEXT:
{context}

USER INPUT:
"{input}"
==============================

🔍 ANALYSIS GOALS:
1. Identify the user's sleep issue, need, or question
2. Extract important data (e.g., time, feelings, habits, metrics)
3. Provide a brief but insightful summary of the user's sleep pattern
4. Detect mood and emotional cues if present
5. Generate a personalized wind-down routine
6. Recommend optimal sleep and wake-up timing
7. Suggest behavior/environment improvements
8. Perform a basic sleep disorder screening
9. Propose follow-up actions and metrics to track
10. Tag context categories (e.g., stress, diet, tech use)

📦 RESPONSE FORMAT (STRICTLY VALID JSON):
Double quotes must be used.
Avoid any markdown formatting or extra text.
Response must end cleanly with the final `}}`.

RESPONSE FORMAT:
{{
  "sleepAssessment": {{
    "issue": "string",  // e.g. 'difficulty falling asleep'
    "confidence": number,  // 0.0 to 1.0
    "severity": "low" | "medium" | "high",
    "summary": "string",
    "sleepHistoryDetected": boolean,
    "userMood": "tired" | "refreshed" | "anxious" | "groggy" | "unknown",
    "sleepDurationTrend": "increasing" | "decreasing" | "stable" | "unknown"
  }},
  "routineRecommendation": {{
    "windDown": ["string"],  // calming activities
    "avoidBeforeBed": ["string"],  // caffeine, screens, etc.
    "optimalSleepTime": "string",  // e.g. "10:45 PM"
    "optimalWakeTime": "string",  // e.g. "6:30 AM"
    "reminders": ["string"]
  }},
  "tips": {{
    "immediateActions": ["string"],  // changes starting tonight
    "lifestyleChanges": ["string"],  // longer-term habits
    "environmentSuggestions": ["string"]  // bedroom, lighting, etc.
  }},
  "recommendations": {{
      "immediate": ["string"], // 3-4 immediate actionable suggestions
      "content": {{
      "spotify": {{
          "genres": ["string"], // ambient, classical, nature sounds, etc.
          "energy": number, // 0-1 (0=calm, 1=energetic)
          "valence": number, // 0-1 (0=sad, 1=happy)
          "mood": "string" // relaxing, uplifting, focus, etc.
      }},
      }}
  }},
  "disorderCheck": {{
    "riskLevel": "low" | "medium" | "high",
    "symptoms": ["string"],
    "recommendations": ["string"],
    "complianceRisk": "low" | "medium" | "high"  // likelihood user may not follow
  }},
  "followUp": {{
    "checkInPeriod": "daily" | "weekly" | "custom",
    "trackMetrics": ["string"],  // e.g. "hours slept", "interruptions"
    "goals": ["string"]  // e.g. "reduce latency to < 20 minutes"
  }},
  "metadata": {{
    "wordCount": number,
    "timeOfDayMentioned": "morning" | "afternoon" | "evening" | "night" | "unknown",
    "contextTags": ["string"]  // e.g. ["stress", "caffeine", "screen use"]
  }}
}}
            GUIDELINES:
            - Be empathetic and non-judgmental
            - Focus on actionable insights
            - Consider cultural sensitivity
            - If serious mental health concerns are detected, prioritize professional help recommendations
            - Tailor content suggestions to the specific mood and needs identified
            - Use clear, supportive language
            - Avoid medical diagnosis or treatment advice
            - Encourage professional help when appropriate

           [SYSTEM RULES]
            1. Respond **ONLY** with valid JSON that starts with `{{` and ends with `}}`.
            2. **Never** include:
            - Text before `{{` or after `}}`
            - Markdown/code blocks (```json```)
            - Explanatory comments
            3. If you violate this, the API will reject your response.

            [EXAMPLE OF CORRECT RESPONSE]
            {{
                "sleepAssessment": {{
                    "issue": "Difficulty falling asleep",
                    "confidence": 0.8
                }}
            }}

            [EXAMPLE OF INCORRECT RESPONSE]
            USER INPUT: "{input}"\n
            {{
                "sleepAssessment": {{
                    "issue": "Difficulty falling asleep"
                }}
            }}

            Return only the JSON object with no additional text or formatting.
"""
,
    input_variables=["context", "input"]
)
        from langchain.chains import LLMChain
        self.chain = LLMChain(prompt=self.prompt, llm=self.llm, verbose=False)

        # self.chain = self.prompt | self.llm
        self.collection = self._get_collection()

    def _get_collection(self):
        # You can customize this to use a DB or tracking log if needed
        return []

    from langchain_core.messages import HumanMessage
    import json

    def get_spotify_recommendation(self, spotify_recommendation: dict):
        try:
            # Step 1: Build structured prompt
            genres = spotify_recommendation.get("genres", [])
            mood = spotify_recommendation.get("mood", "")
            energy = spotify_recommendation.get("energy", "")
            valence = spotify_recommendation.get("valence", "")

            prompt = (
                "Find a Spotify playlist or track recommendation with the following:\n"
                f"- Genres: {', '.join(genres)}\n"
                f"- Mood: {mood}\n"
                f"- Energy level: {energy} (0=low, 1=high)\n"
                f"- Valence (positivity): {valence} (0=sad, 1=happy)\n\n"
                "Only return JSON with: title, spotify_url, duration_seconds, thumbnail, and artist/playlist details."
            )
            # print("🎵 Prompt to Spotify agent:\n", prompt)

            # Step 2: Call the Spotify agent
            response = spotify_assistant.invoke({
                "messages": [HumanMessage(content=prompt)]
            })

            # Step 3: Extract the last message's content
            content = response["messages"][-1].content
            # print("📦 Raw content from Spotify assistant:\n", content)

            # Step 4: Parse the JSON content
            if isinstance(content, dict):
                return content  # already a dict

            if isinstance(content, str):
                try:
                    return json.loads(content)
                except json.JSONDecodeError as e:
                    return {"error": f"Failed to parse JSON: {str(e)}"}

            return {"error": "Unknown content format from Spotify assistant."}

        except Exception as e:
            return {"error": f"Exception occurred: {str(e)}"}

    def run(self, user_input, user_id=None, context=None):
        try:
            context_str = """
                - MindFuel is a mental wellness app that helps users track mood, get personalized content recommendations, and improve mental health
                - Users share their thoughts, feelings, and current state
                - Your analysis will be used to provide personalized YouTube videos, articles, Spotify playlists, and meditation recommendations
                - Be empathetic, supportive, and professional in your analysis
            """
            prompt_vars = {
                "context": context_str,
                "input": user_input,
            }

            result = self.chain.invoke(prompt_vars)

            import re, json

            # Extract the actual JSON string from the 'text' key
            if isinstance(result, dict) and "text" in result:
                result_text = result["text"]
            else:
                result_text = result

            # Parse result_text
            if isinstance(result_text, str):
                try:
                    parsed_result = json.loads(result_text.strip())
                except json.JSONDecodeError:
                    json_str = re.search(r"\{[\s\S]*\}", result_text).group(0)
                    parsed_result = json.loads(json_str)
            elif isinstance(result_text, dict):
                parsed_result = result_text
            else:
                raise ValueError("Unexpected result type.")

            # ✅ Now this should work:
            spotify_rec = parsed_result.get("recommendations", {}).get("content", {}).get("spotify")
            if spotify_rec:
                spotify_playlist = self.get_spotify_recommendation(spotify_rec)
                # print("🎵 Spotify playlist result:\n", json.dumps(spotify_playlist, indent=2))
                parsed_result["recommendations"]["content"]["spotify"]["playlist"] = spotify_playlist

            return parsed_result

        except Exception as e:
            print("❌ Run error:", e)
            return {"error": str(e)}





In [ ]:
# #spotify missing
# def run(self, user_input, user_id=None, context=None):
#         try:
#             context_str = """
#                 - MindFuel is a mental wellness app that helps users track mood, get personalized content recommendations, and improve mental health
#                 - Users share their thoughts, feelings, and current state
#                 - Your analysis will be used to provide personalized YouTube videos, articles, Spotify playlists, and meditation recommendations
#                 - Be empathetic, supportive, and professional in your analysis
#             """
#             prompt_vars = {
#                 "context": context_str,
#                 "input": user_input,
#             }

#             import re, json


#             result = self.chain.invoke(prompt_vars)

#             if isinstance(result, dict):
#               parsed_result = result  # already parsed, no need to regex or load again

#               # Process Spotify recommendation
#               spotify_rec = parsed_result.get("recommendations", {}).get("content", {}).get("spotify")
#               if spotify_rec:
#                   spotify_playlist = self.get_spotify_recommendation(spotify_rec)
#                   print("🎵 Spotify playlist result:\n", json.dumps(spotify_playlist, indent=2))  # ✅ print playlist
#                   parsed_result["recommendations"]["content"]["spotify"]["playlist"] = spotify_playlist

#               return parsed_result


#         except Exception as e:
#             print("❌ Run error:", e)
#             return {"error": str(e)}

In [40]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I have trouble falling asleep and usually spend over an hour awake in bed. I also drink coffee late in the afternoon.")
print(json.dumps(response, indent=2))


{
  "sleepAssessment": {
    "issue": "Difficulty falling asleep",
    "confidence": 0.9,
    "severity": "medium",
    "summary": "The user experiences difficulty falling asleep and spends over an hour awake in bed, possibly due to late afternoon coffee consumption.",
    "sleepHistoryDetected": true,
    "userMood": "anxious",
    "sleepDurationTrend": "decreasing"
  },
  "routineRecommendation": {
    "windDown": [
      "Reading a book",
      "Practicing deep breathing exercises"
    ],
    "avoidBeforeBed": [
      "Coffee",
      "Screen time"
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:00 AM",
    "reminders": [
      "Create a calming bedtime routine"
    ]
  },
  "tips": {
    "immediateActions": [
      "Limit caffeine intake after noon",
      "Establish a consistent sleep schedule"
    ],
    "lifestyleChanges": [
      "Incorporate relaxation techniques into daily routine",
      "Create a sleep-conducive environment"
    ],
    "environmentSugges

In [39]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I wake up several times during the night and feel anxious before going to bed.")
print(json.dumps(response, indent=2))


{
  "sleepAssessment": {
    "issue": "Frequent awakenings and bedtime anxiety",
    "confidence": 0.9,
    "severity": "medium",
    "summary": "User experiences anxiety before bedtime and wakes up multiple times during the night.",
    "sleepHistoryDetected": true,
    "userMood": "anxious",
    "sleepDurationTrend": "unknown"
  },
  "routineRecommendation": {
    "windDown": [
      "Reading a calming book",
      "Deep breathing exercises",
      "Listening to soothing music"
    ],
    "avoidBeforeBed": [
      "Caffeine intake",
      "Screen time",
      "Strenuous exercise"
    ],
    "optimalSleepTime": "10:00 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Create a relaxing bedtime routine",
      "Maintain a consistent sleep schedule"
    ]
  },
  "tips": {
    "immediateActions": [
      "Try progressive muscle relaxation before bed",
      "Create a calming sleep environment"
    ],
    "lifestyleChanges": [
      "Incorporate mindfulness meditation into d

In [37]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
print(json.dumps(response, indent=2))


{
  "sleepAssessment": {
    "issue": "Feeling tired in the morning despite 8 hours of sleep and phone use before bed",
    "confidence": 0.7,
    "severity": "medium",
    "summary": "User experiences morning tiredness and may have a disruptive bedtime routine with phone usage.",
    "sleepHistoryDetected": true,
    "userMood": "tired",
    "sleepDurationTrend": "stable"
  },
  "routineRecommendation": {
    "windDown": [
      "Reading a book",
      "Listening to calming music",
      "Meditation"
    ],
    "avoidBeforeBed": [
      "Phone usage"
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Set a bedtime alarm for wind-down routine"
    ]
  },
  "tips": {
    "immediateActions": [
      "Limit phone usage before bed",
      "Create a relaxing bedtime routine"
    ],
    "lifestyleChanges": [
      "Establish a consistent sleep schedule",
      "Create a comfortable sleep environment"
    ],
    "environmentSuggestions": [
 

In [29]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
print(json.dumps(response, indent=2))


{
  "sleepAssessment": {
    "issue": "feeling tired despite sufficient sleep",
    "confidence": 0.85,
    "severity": "medium",
    "summary": "You are getting 8 hours of sleep, which is generally considered sufficient, but you still feel tired in the morning. This may be influenced by your use of your phone before bed.",
    "sleepHistoryDetected": false,
    "userMood": "tired",
    "sleepDurationTrend": "stable"
  },
  "routineRecommendation": {
    "windDown": [
      "Read a book",
      "Practice deep breathing",
      "Listen to calming music"
    ],
    "avoidBeforeBed": [
      "Phone usage",
      "Caffeine"
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Set a bedtime alarm",
      "Limit screen time 30 minutes before bed"
    ]
  },
  "tips": {
    "immediateActions": [
      "Put your phone away 30 minutes before bed",
      "Try reading or meditating before sleep"
    ],
    "lifestyleChanges": [
      "Establish a 

In [14]:
agent = SleepCoachAgent(api_key=key)
response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
print(json.dumps(response, indent=2))




> Entering new LLMChain chain...
Prompt after formatting:

You are a smart, caring, and proactive Sleep Coach AI. Your role is to help users reflect on their sleep, understand behavioral patterns, and build sustainable habits for better rest and recovery.

Your job is to analyze the user's sleep-related input and provide a rich JSON response structured across multiple categories.

CONTEXT:

                - MindFuel is a mental wellness app that helps users track mood, get personalized content recommendations, and improve mental health
                - Users share their thoughts, feelings, and current state
                - Your analysis will be used to provide personalized YouTube videos, articles, Spotify playlists, and meditation recommendations
                - Be empathetic, supportive, and professional in your analysis
            

USER INPUT:
"I sleep 8 hours but still feel tired in the morning. I often use my phone before bed."

🔍 ANALYSIS GOALS:
1. Identify the user's s

In [ ]:
!pip install   langsmith


In [ ]:
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# # os.environ["LANGCHAIN_API_KEY"] = "key"
# os.environ["LANGCHAIN_PROJECT"] = "sleep-coach"
# # os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_API_KEY="key"


In [ ]:
# agent = SleepCoachAgent(api_key=key)
# response = agent.run("I sleep 8 hours but still feel tired in the morning. I often use my phone before bed.")
# print(json.dumps(response, indent=2))


{
  "sleepAssessment": {
    "issue": "feeling tired despite adequate sleep",
    "confidence": 0.85,
    "severity": "medium",
    "summary": "You are getting 8 hours of sleep, but the use of your phone before bed may be affecting your sleep quality, leading to feelings of tiredness in the morning.",
    "sleepHistoryDetected": false,
    "userMood": "tired",
    "sleepDurationTrend": "stable"
  },
  "routineRecommendation": {
    "windDown": [
      "read a book",
      "practice deep breathing",
      "listen to calming music"
    ],
    "avoidBeforeBed": [
      "phone usage",
      "screen time",
      "caffeine"
    ],
    "optimalSleepTime": "10:30 PM",
    "optimalWakeTime": "6:30 AM",
    "reminders": [
      "Set a bedtime alarm",
      "Create a relaxing pre-sleep routine"
    ]
  },
  "tips": {
    "immediateActions": [
      "Put your phone away at least 30 minutes before bed",
      "Try a relaxing activity before sleep"
    ],
    "lifestyleChanges": [
      "Establish a

In [6]:
!pip install -U langchain langchain-openai

In [ ]:
# LANGSMITH_TRACING="true"
# LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
# LANGSMITH_API_KEY="key"
# LANGSMITH_PROJECT="sleep_agent"


In [ ]:
# from langchain_openai import ChatOpenAI
# from langsmith import traceable
# from langsmith.wrappers import wrap_openai

# openai_client = wrap_openai(OpenAI())
# llm = ChatOpenAI(api_key=key, model="gpt-4o-mini")
# llm.invoke("Hello, world!")

AttributeError: 'ChatOpenAI' object has no attribute 'chat'

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]="key"
os.environ["LANGCHAIN_PROJECT"]="Sleep"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

In [ ]:
# key

In [11]:
print("✅ Tracing enabled:", os.environ.get("LANGCHAIN_TRACING_V2"))


✅ Tracing enabled: true
